In [19]:
from dotenv import load_dotenv
import os

# .env load 
load_dotenv()

# check API KEY 
if "OPENAI_API_KEY" not in os.environ:
    print("API KEY 정보가 없습니다. 확인 후 환경변수에 등록해주세요.")

In [16]:
# ImportError -> pip install -U docarray로 해결
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough 
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["철수는 유리를 좋아한다.", "유리는 훈이를 좋아한다."],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """{context}기반으로 question에 답해주세요.

question: {question}"""

prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser=StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser 

response1 = chain.invoke("철수는 누구를 좋아해?")
response2 = chain.invoke("유리는 누구를 좋아해?")
print(response1)
print(response2)

철수는 유리를 좋아해요.
훈이를 좋아한다.
